# Block

Mizore uses `Block` to arrange the quantum operations and provide intrinsic support of parameterized circuits together with `BlockCircuit`.

## Basic Usage

Please see [BasicConcepts](BasicConcepts.ipynb) for a basic instruction.

##  Blocks Provided

We tends to call a block *entangler* if it can entangle the qubits it acts on. 

- Rotation Entangler $e^{iPt}$
- Multi-Rotation Entangler $e^{iP_1 t_1} e^{iP_2 t_2} .. e^{iP_n t_n}$
- Single Parameter Multi-Rotation Entangler $e^{i P_1 a_1 t} e^{iP_2 a_2 t} .. e^{iP_n a_n t}$
- Efficient Coupled Cluster
- Hardware Efficient Entangler
- Hartree-Fock Initial Block
- Pauli Gates Block
- Time Evolution Block
- Compositive Block

We will apply the blocks to $|0000\rangle$ to demonstrate their performances:

In [3]:
from Utilities.CircuitEvaluation import get_quantum_engine
from projectq.ops import All, Measure
n_qubit=4
compiler_engine = get_quantum_engine()
wavefunction = compiler_engine.allocate_qureg(n_qubit) # Initialize the wavefunction
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))
All(Measure) | wavefunction
compiler_engine.flush()

The amplitude of |0000>: (1+0j)


### Rotation Entanglers
`RotationEntangler` block is a time evolution operator of a single Pauli string $e^{iPt}$ which can be applied to the wavefunction, where time t is an adjustable parameter.

In [2]:
from Blocks import RotationEntangler
n_qubit=4
compiler_engine = get_quantum_engine()
wavefunction = compiler_engine.allocate_qureg(n_qubit) # Initialize the wavefunction
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))

entangler=RotationEntangler((1,2,3),(3,2,1))
print("The entangler to apply:")
print(entangler)
entangler.apply([0.5],wavefunction)
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))
entangler.is_inversed=True
print(entangler)
entangler.apply([0.5],wavefunction)
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))

All(Measure) | wavefunction
compiler_engine.flush()

(Note: This is the (slow) Python simulator.)
The amplitude of |0000>: (1+0j)
The entangler to apply:
Type:RotationEntangler; Para Num:1; Qsubset:(1, 2, 3); Pauli:ZYX
The amplitude of |0000>: (0.8775825618903725+0j)
Type:RotationEntangler; Para Num:1; INVERSED; Qsubset:(1, 2, 3); Pauli:ZYX
The amplitude of |0000>: (0.9999999999999998+0j)


`MultiRotationEntangler` block is a series of time evolution operators of Pauli strings $e^{iP_1 t_1} e^{iP_2 t_2} .. e^{iP_n t_n}$ which can be applied to the wavefunction, where time $t_1,...,t_n$ are all adjustable parameters.

In [3]:
from Blocks import MultiRotationEntangler
from openfermion.ops import QubitOperator
n_qubit=4
compiler_engine = get_quantum_engine()
wavefunction = compiler_engine.allocate_qureg(n_qubit) # Initialize the wavefunction
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))

entangler = MultiRotationEntangler(0.3*QubitOperator("X" + str(0) + " Y" + str(1))
            +0.5*QubitOperator("X" + str(1) + " Y" + str(2)), init_angle = [0.3,0.6])
print("The entangler to apply:")
print(entangler)
entangler.apply([1.0,1.0], wavefunction)
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))

All(Measure) | wavefunction
compiler_engine.flush()

(Note: This is the (slow) Python simulator.)
The amplitude of |0000>: (1+0j)
The entangler to apply:
Type:MultiRotationEntangler; Para Num:2; N Rotation:2; [0.3, 0.6]
The amplitude of |0000>: (-0.0078108380119922705+0j)
Type:MultiRotationEntangler; Para Num:2; INVERSED; N Rotation:2; [0.3, 0.6]
The amplitude of |0000>: (0.9999999999999992+0j)


`SingleParameterMultiRotationEntangler` block is a series of time evolution operators of Pauli strings $e^{iP_1 t} e^{iP_2 t} .. e^{iP_n t}$ which can be applied to the wavefunction, where time t is a single adjustable parameters.

In [4]:
from Blocks import SingleParameterMultiRotationEntangler
from openfermion.ops import QubitOperator
n_qubit=4
compiler_engine = get_quantum_engine()
wavefunction = compiler_engine.allocate_qureg(n_qubit) # Initialize the wavefunction
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))

entangler = SingleParameterMultiRotationEntangler(0.3*QubitOperator("X" + str(0) + " Y" + str(1))
            +0.5*QubitOperator("X" + str(1) + " Y" + str(2)), init_angle = [0.5])
print("The entangler to apply:")
print(entangler)
entangler.apply([1.0], wavefunction)
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))

All(Measure) | wavefunction
compiler_engine.flush()

(Note: This is the (slow) Python simulator.)
The amplitude of |0000>: (1+0j)
The entangler to apply:
Type:SingleParameterMultiRotationEntangler; Para Num:1; N Rotation:2; [0.5]
The amplitude of |0000>: (0.6588471218011396+0j)
Type:SingleParameterMultiRotationEntangler; Para Num:1; INVERSED; N Rotation:2; [0.5]
The amplitude of |0000>: (0.9999999999999996+0j)


### Efficient Coupled Cluster
`EfficientCoupledCluster` block can apply a circuit similar to Coupled Cluster operator to wavefunction.

Apply $\sum_{i=1}^n R_x(t_{2*i})Ry(t_{2*i-1}), e^{iX_1...X_n t_0}, \sum_{i=1}^n R_y(-t_{2*i-1})R_x(-t_{2*i})$  rotation to n qubits with different angle 

One can specify the 


In [5]:
from Blocks import EfficientCoupledCluster
n_qubit=4
compiler_engine = get_quantum_engine()
wavefunction = compiler_engine.allocate_qureg(n_qubit) # Initialize the wavefunction
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))

qsubset = [0,1,2,3]
entangler = EfficientCoupledCluster(qsubset)
print("The entangler to apply:")
print(entangler)
entangler.apply([1.0 for i in range(4*2+1)], wavefunction)
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))

All(Measure) | wavefunction
compiler_engine.flush()

(Note: This is the (slow) Python simulator.)
The amplitude of |0000>: (1+0j)
The entangler to apply:
Type:EfficientCoupledCluster; Para Num:9; Qsubset:[0, 1, 2, 3]
The amplitude of |0000>: (0.5403023058681544-0.03595365205585828j)
Type:EfficientCoupledCluster; Para Num:9; INVERSED; Qsubset:[0, 1, 2, 3]
The amplitude of |0000>: (0.9999999999999997-3.3660741681952025e-14j)


### Hardware efficient entangler
`HardwareEfficientEntangler` block will apply the circuit of Hardware-efficient ansatz proposed in [Nature 549, 242–246(2017)](https://www.nature.com/articles/nature23879?sf114016447=1) to wavefunction.

In [6]:
from Blocks import HardwareEfficientEntangler
from openfermion.ops import QubitOperator
n_qubit=4
compiler_engine = get_quantum_engine()
wavefunction = compiler_engine.allocate_qureg(n_qubit) # Initialize the wavefunction
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))

qsubset = [0,1,2,3]
entangler = HardwareEfficientEntangler(qsubset)
print("The entangler to apply:")
print(entangler)
entangler.apply([1.0 for i in range(3*len(qsubset))], wavefunction)
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))

All(Measure) | wavefunction
compiler_engine.flush()

(Note: This is the (slow) Python simulator.)
The amplitude of |0000>: (1+0j)
The entangler to apply:
Type:HardwareEfficientEntangler; Para Num:12; Qsubset:[0, 1, 2, 3]
The amplitude of |0000>: (-0.20668023962046486+0.004565812957204471j)
Type:HardwareEfficientEntangler; Para Num:12; INVERSED; Qsubset:[0, 1, 2, 3]
The amplitude of |0000>: (1.0000000000000009+4.31464595980615e-13j)


### Hartree-Fock Initial Block
`HartreeFockInitBlock` can apply $X$ gates on a few of qubits. Usually for getting Hartree-Fock qubit wavefunction.

In [7]:
from Blocks import HartreeFockInitBlock
n_qubit=4
compiler_engine = get_quantum_engine()
wavefunction = compiler_engine.allocate_qureg(n_qubit) # Initialize the wavefunction
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))

qsubset = [0,1]
entangler = HartreeFockInitBlock(qsubset)
print("The entangler to apply:")
print(entangler)
entangler.apply([1.0], wavefunction)
compiler_engine.flush()
print("The amplitude of |1100>:",compiler_engine.backend.get_amplitude([1,1,0,0], wavefunction))

All(Measure) | wavefunction
compiler_engine.flush()

(Note: This is the (slow) Python simulator.)
The amplitude of |0000>: (1+0j)
The entangler to apply:
Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0, 1]
The amplitude of |1100>: (1+0j)
Type:HartreeFockInitBlock; Para Num:0; INVERSED; Qsubset:[0, 1]
The amplitude of |0000>: (1+0j)


### Pauli Gates Block
`PauliGatesBlock` can apply Pauli gates represented by paulistrings directly to the wavefunction, where paulistring should be like \[(0,'X'),(2,'Y')\]

In [8]:
from Blocks import PauliGatesBlock
n_qubit=4
compiler_engine = get_quantum_engine()
wavefunction = compiler_engine.allocate_qureg(n_qubit) # Initialize the wavefunction
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))

paulistring = [(0,'X'),(2,'X')]
entangler = PauliGatesBlock(paulistring)
print("The entangler to apply:")
print(entangler)
entangler.apply([1.0], wavefunction)
compiler_engine.flush()
print("The amplitude of |1010>:",compiler_engine.backend.get_amplitude([1,0,1,0], wavefunction))

All(Measure) | wavefunction
compiler_engine.flush()

(Note: This is the (slow) Python simulator.)
The amplitude of |0000>: (1+0j)
The entangler to apply:
Type:PauliGatesBlock; Para Num:0; PauliString:[(0, 'X'), (2, 'X')]
The amplitude of |1010>: (1+0j)
Type:PauliGatesBlock; Para Num:0; INVERSED; PauliString:[(0, 'X'), (2, 'X')]
The amplitude of |0000>: (1+0j)


### Time Evolution Block
`TimeEvolutionBlock` can apply time evolution operator $e^{iHt}$ to the wavefunction, where $H$ is the hamiltonian(`QubitOperator`)

In [9]:
from Blocks import TimeEvolutionBlock
from openfermion.ops import QubitOperator
n_qubit=4
compiler_engine = get_quantum_engine()
wavefunction = compiler_engine.allocate_qureg(n_qubit) # Initialize the wavefunction
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))

hamiltonian = 0.3*QubitOperator("X" + str(0) + " Y" + str(1)) + 0.5*QubitOperator("X" + str(1) + " Y" + str(2))
entangler = TimeEvolutionBlock(hamiltonian)
print("The entangler to apply:")
print(entangler)
entangler.apply([0.5], wavefunction)
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))

All(Measure) | wavefunction
compiler_engine.flush()

(Note: This is the (slow) Python simulator.)
The amplitude of |0000>: (1+0j)
The entangler to apply:
Type:TimeEvolutionBlock; Para Num:1; TimeEvolution: T=0
The amplitude of |0000>: (0.9578001900087135+0j)
Type:TimeEvolutionBlock; Para Num:1; INVERSED; TimeEvolution: T=0
The amplitude of |0000>: (0.9999999999999987+0j)


### Compositive Block

`CompositiveBlock` is a `Block` defined by a `BlockCircuit`.

In the following example, a `BlockCircuit` is constructed and converted into a `CompositiveBlock`.

In [17]:
from Blocks import PauliGatesBlock, TimeEvolutionBlock, BlockCircuit, CompositiveBlock
from openfermion.ops import QubitOperator

compiler_engine = get_quantum_engine()
wavefunction = compiler_engine.allocate_qureg(n_qubit)
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))

entangler1 = PauliGatesBlock([(0,'X'),(2,'X')])
hamiltonian = 0.1*QubitOperator("X0 Y1") + 0.1*QubitOperator("X1 Y2")
entangler2 = TimeEvolutionBlock(hamiltonian,init_angle=0.5)

circuit = BlockCircuit(3)
circuit.add_block(entangler1)
circuit.add_block(entangler2)

compositive_block = CompositiveBlock(circuit)
compositive_block.apply([1.0, 1.0], wavefunction)

compiler_engine.flush()
print("The amplitude of |1010>:",compiler_engine.backend.get_amplitude([1,0,1,0], wavefunction))

All(Measure) | wavefunction
compiler_engine.flush()

The amplitude of |0000>: (1+0j)
The amplitude of |1010>: (0.9775842485391512+0j)
